In [ ]:
import numpy as np
import pandas as pd

In [ ]:
dataset_dir = "./Plant_Village_Data_Set/"

In [ ]:
dataset_dir

In [ ]:
from sklearn.model_selection import train_test_split
import os

# List all subdirectories (each corresponds to a disease category)
subdirectories = [subdir for subdir in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, subdir))]

# Create lists to store image paths and corresponding labels
image_paths = []
labels = []
sum = 0
# Iterate through each subdirectory (disease category)
for subdir in subdirectories:
    subdir_path = os.path.join(dataset_dir, subdir)
    
    subdir_images = [os.path.join(subdir_path, img) for img in os.listdir(subdir_path) if img.lower().endswith('.jpg')]

    sum += (len(subdir_images))
    image_paths.extend(subdir_images)
    labels.extend([subdir] * len(subdir_images))
print(sum)
# Split the data into training and testing sets
train_paths, test_paths, train_labels, test_labels = train_test_split(image_paths, labels, test_size=0.2, random_state=42)


In [ ]:
subdirectories

In [ ]:
test_labels

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical
def load_images(image_paths):
    images = []
    for path in image_paths:
        image = cv2.imread(path)  # Load image using OpenCV
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        images.append(image)
    return np.array(images)

# Load training and testing images
train_images = load_images(train_paths)
test_images = load_images(test_paths)

# Convert labels to categorical format (if using one-hot encoding)
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

num_classes = len(np.unique(train_labels_encoded))
train_labels_onehot = to_categorical(train_labels_encoded, num_classes=num_classes)
test_labels_onehot = to_categorical(test_labels_encoded, num_classes=num_classes)


In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img
from tensorflow.keras.utils import to_categorical

# Preprocess images (normalize pixel values and resize)
def preprocess_images(images, target_size=(224, 224)):
    processed_images = []
    for img in images:
        img = img.astype('float32') / 255.0  # Normalize pixel values
        img = cv2.resize(img, target_size)   # Resize image
        processed_images.append(img)
    return np.array(processed_images)

# Preprocess training and testing images
train_images_processed = preprocess_images(train_images)
test_images_processed = preprocess_images(test_images)

# Check shapes of preprocessed data
print("Train Images Shape:", train_images_processed.shape)
print("Test Images Shape:", test_images_processed.shape)
print("Train Labels Shape:", train_labels_onehot.shape)
print("Test Labels Shape:", test_labels_onehot.shape)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Define CNN model architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(train_images_processed, train_labels_onehot,
                    batch_size=32, epochs=10, validation_split=0.2)


In [ ]:
# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(test_images_processed, test_labels_onehot)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


In [ ]:
# Make predictions on test data
predictions = model.predict(test_images_processed)

import numpy as np

# Convert predictions to class labels
predicted_labels = np.argmax(predictions, axis=1)


from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Compute confusion matrix
cm = confusion_matrix(test_labels_encoded, predicted_labels)

# Display the confusion matrix
classes = label_encoder.classes_
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)

# Plot confusion matrix

